# Running Native and Semantic Functions Inline


* Let's dynamically generate the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

For now we'll use only the `{{$input}}` variable, and see more complex templates later.


Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.Plugins.Core;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.Globalization;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Plugins.Core, 1.2.0-alpha

create a semantic function used to summarize content, be a pirate:

In [3]:
// Specify and get the value type as generic parameter

#!import plugins/StaticTextPlugin.cs

var textPlugin = kernel.ImportPluginFromType<StaticTextPlugin>();

var args = new KernelArguments()
        {
            ["input"] = "Today is: ",
            ["day"] = DateTimeOffset.Now.ToString("dddd", CultureInfo.CurrentCulture)
        };
string? resultValue = await kernel.InvokeAsync<string>(textPlugin["AppendDay"], args);
Console.WriteLine($"string -> {resultValue}");

string -> Today is: Saturday


In [4]:
var args = new KernelArguments()
        {
            ["input"] = resultValue,
        };

FunctionResult functionResult = await kernel.InvokeAsync(textPlugin["Uppercase"], args);

Console.WriteLine($"string -> {functionResult}");

string -> TODAY IS: SATURDAY


Load native plugin into the kernel function collection, sharing its functions with prompt templates

In [5]:
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<TimePlugin>("time");

In [6]:



// Prompt Function invoking time.Date and time.Time method functions
const string FunctionDefinition = @"
Today is: {{time.Date}}
Current time is: {{time.Time}}

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropirate?
";

// This allows to see the prompt before it's sent to OpenAI
Console.WriteLine("--- Rendered Prompt");
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(new PromptTemplateConfig(FunctionDefinition));
var renderedPrompt = await promptTemplate.RenderAsync(kernel);
Console.WriteLine(renderedPrompt);

// Run the prompt / prompt function
var kindOfDay = kernel.CreateFunctionFromPrompt(FunctionDefinition, new OpenAIPromptExecutionSettings() { MaxTokens = 100 });


--- Rendered Prompt

Today is: Saturday, 27 January 2024
Current time is: 01:45:30 PM

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropirate?



In [7]:
// Run the prompt / prompt function
var kindOfDay = kernel.CreateFunctionFromPrompt(FunctionDefinition, new OpenAIPromptExecutionSettings() { MaxTokens = 100 });


In [8]:
 // Show the result
Console.WriteLine("--- Prompt Function result");
var result = await kernel.InvokeAsync(kindOfDay);
Console.WriteLine(result.GetValue<string>());

--- Prompt Function result
```json
{
  "current_time": "01:45:36 PM",
  "date": "Saturday, 27 January 2024",
  "time_of_day": "afternoon",
  "weekend_status": "weekend",
  "appropriate_cocktail": "Mimosa"
}
```


In [9]:
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"   {func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}



In [10]:
var functions = kernel.Plugins.GetFunctionsMetadata();       
foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: StaticTextPlugin
   Uppercase: Change all string chars to uppercase
      Params:
      - input: Text to uppercase
        default: ''

Plugin: StaticTextPlugin
   AppendDay: Append the day variable
      Params:
      - input: Text to append to
        default: ''
      - day: Value of the day to append
        default: ''

Plugin: time
   Date: Get the current date

Plugin: time
   Today: Get the current date

Plugin: time
   Now: Get the current date and time in the local time zone

Plugin: time
   UtcNow: Get the current UTC date and time

Plugin: time
   Time: Get the current time

Plugin: time
   Year: Get the current year

Plugin: time
   Month: Get the current month name

Plugin: time
   MonthNumber: Get the current month number

Plugin: time
   Day: Get the current day of the month

Plugin: time
   DaysAgo: Get the date offset by a provided number of days from today
      Params:
      - input: The number of days to offset from today
        default: ''

Plugin: time
 